## Emoji prediction for weibo tweets
This is the final project for Nanjing University Data Mining class in spring 2019

In [10]:
import pandas as pd

Load the raw weibo tweets (train.text) and preprocessed label (train.label). Save them into two lists respectively.

In [15]:
DATA_FOLDER='data/'
TRAIN_TEXT='train.text'
TRAIN_LABEL='train.label'
N_ROWS=10
text_list = []
label_list = []
with open(DATA_FOLDER+TRAIN_TEXT, 'r') as tweets, open(DATA_FOLDER+TRAIN_LABEL) as labels:
    for tweet in tweets:
        tweet = tweet.replace('\'', '')
        tweet = tweet.replace('\"', '')
        text_list.append(tweet.replace('\n', ''))  # the last char is \n, so we just ignore it
    for label in labels:  
        label_list.append(int(label))  # same as before, ignore \n
    assert(len(text_list)==len(label_list))

Save it to a dataframe and csv file for future use. Use '|' as a delimeter to minimize the chance of mistaking a punctuation in tweet as one.

In [16]:
train_df = pd.DataFrame({'tweet':text_list, 'label':label_list})
train_df.to_csv(DATA_FOLDER+'train.csv', index=False, sep='|')

In [4]:
# this is a very useful data structure from Berkeley CS188's project, in util.py
# -------
# Licensing Information:  You are free to use or extend these projects for
# educational purposes provided that (1) you do not distribute or publish
# solutions, (2) you retain this notice, and (3) you provide clear
# attribution to UC Berkeley, including a link to http://ai.berkeley.edu.
# 
# Attribution Information: The Pacman AI projects were developed at UC Berkeley.
# The core projects and autograders were primarily created by John DeNero
# (denero@cs.berkeley.edu) and Dan Klein (klein@cs.berkeley.edu).
# Student side autograding was added by Brad Miller, Nick Hay, and
# Pieter Abbeel (pabbeel@cs.berkeley.edu).
class Counter(dict):
    """
    A counter keeps track of counts for a set of keys.
    The counter class is an extension of the standard python
    dictionary type.  It is specialized to have number values
    (integers or floats), and includes a handful of additional
    functions to ease the task of counting data.  In particular,
    all keys are defaulted to have value 0.  Using a dictionary:
    a = {}
    print a['test']
    would give an error, while the Counter class analogue:
    >>> a = Counter()
    >>> print a['test']
    0
    returns the default 0 value. Note that to reference a key
    that you know is contained in the counter,
    you can still use the dictionary syntax:
    >>> a = Counter()
    >>> a['test'] = 2
    >>> print a['test']
    2
    This is very useful for counting things without initializing their counts,
    see for example:
    >>> a['blah'] += 1
    >>> print a['blah']
    1
    The counter also includes additional functionality useful in implementing
    the classifiers for this assignment.  Two counters can be added,
    subtracted or multiplied together.  See below for details.  They can
    also be normalized and their total count and arg max can be extracted.
    """
    def __getitem__(self, idx):
        self.setdefault(idx, 0)
        return dict.__getitem__(self, idx)

    def incrementAll(self, keys, count):
        """
        Increments all elements of keys by the same count.
        >>> a = Counter()
        >>> a.incrementAll(['one','two', 'three'], 1)
        >>> a['one']
        1
        >>> a['two']
        1
        """
        for key in keys:
            self[key] += count

    def argMax(self):
        """
        Returns the key with the highest value.
        """
        if len(list(self.keys())) == 0: return None
        all = list(self.items())
        values = [x[1] for x in all]
        maxIndex = values.index(max(values))
        return all[maxIndex][0]

    def sortedKeys(self):
        """
        Returns a list of keys sorted by their values.  Keys
        with the highest values will appear first.
        >>> a = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> a['third'] = 1
        >>> a.sortedKeys()
        ['second', 'third', 'first']
        """
        sortedItems = list(self.items())
        compare = lambda x, y:  sign(y[1] - x[1])
        sortedItems.sort(cmp=compare)
        return [x[0] for x in sortedItems]

    def totalCount(self):
        """
        Returns the sum of counts for all keys.
        """
        return sum(self.values())

    def normalize(self):
        """
        Edits the counter such that the total count of all
        keys sums to 1.  The ratio of counts for all keys
        will remain the same. Note that normalizing an empty
        Counter will result in an error.
        """
        total = float(self.totalCount())
        if total == 0: return
        for key in list(self.keys()):
            self[key] = self[key] / total

    def divideAll(self, divisor):
        """
        Divides all counts by divisor
        """
        divisor = float(divisor)
        for key in self:
            self[key] /= divisor

    def copy(self):
        """
        Returns a copy of the counter
        """
        return Counter(dict.copy(self))

    def __mul__(self, y ):
        """
        Multiplying two counters gives the dot product of their vectors where
        each unique label is a vector element.
        >>> a = Counter()
        >>> b = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> b['first'] = 3
        >>> b['second'] = 5
        >>> a['third'] = 1.5
        >>> a['fourth'] = 2.5
        >>> a * b
        14
        """
        sum = 0
        x = self
        if len(x) > len(y):
            x,y = y,x
        for key in x:
            if key not in y:
                continue
            sum += x[key] * y[key]
        return sum

    def __radd__(self, y):
        """
        Adding another counter to a counter increments the current counter
        by the values stored in the second counter.
        >>> a = Counter()
        >>> b = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> b['first'] = 3
        >>> b['third'] = 1
        >>> a += b
        >>> a['first']
        1
        """
        for key, value in list(y.items()):
            self[key] += value

    def __add__( self, y ):
        """
        Adding two counters gives a counter with the union of all keys and
        counts of the second added to counts of the first.
        >>> a = Counter()
        >>> b = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> b['first'] = 3
        >>> b['third'] = 1
        >>> (a + b)['first']
        1
        """
        addend = Counter()
        for key in self:
            if key in y:
                addend[key] = self[key] + y[key]
            else:
                addend[key] = self[key]
        for key in y:
            if key in self:
                continue
            addend[key] = y[key]
        return addend

    def __sub__( self, y ):
        """
        Subtracting a counter from another gives a counter with the union of all keys and
        counts of the second subtracted from counts of the first.
        >>> a = Counter()
        >>> b = Counter()
        >>> a['first'] = -2
        >>> a['second'] = 4
        >>> b['first'] = 3
        >>> b['third'] = 1
        >>> (a - b)['first']
        -5
        """
        addend = Counter()
        for key in self:
            if key in y:
                addend[key] = self[key] - y[key]
            else:
                addend[key] = self[key]
        for key in y:
            if key in self:
                continue
            addend[key] = -1 * y[key]
        return addend

In [5]:
label_count = Counter()

with open(DATA_FOLDER+TRAIN_LABEL, 'r') as labels:
    for label in labels:
        label_count[int(label)] += 1
print(label_count)
label_count.normalize()
argmax = label_count.argMax()
threshold = 20
for label in label_count:
    cnt = 1 / (label_count[label] / label_count[argmax])
    label_count[label] = cnt if cnt < threshold else threshold
    
assert(len(label_count)==72)
weight_list = [0] * len(label_count)
for label in label_count:
    weight_list[label]=label_count[label]
weight_list

{0: 4094, 1: 13076, 2: 6713, 3: 74788, 4: 2467, 5: 3645, 6: 13152, 7: 2631, 8: 6658, 9: 9465, 10: 15268, 11: 4885, 12: 35155, 13: 6703, 15: 24284, 16: 22381, 17: 10171, 18: 15084, 19: 12825, 14: 46816, 20: 5230, 21: 3721, 22: 6503, 23: 25480, 24: 11867, 25: 3108, 27: 16762, 29: 47561, 30: 19735, 31: 32172, 32: 8488, 28: 11351, 34: 47628, 37: 9423, 38: 6854, 26: 5979, 39: 6948, 40: 10505, 41: 10981, 42: 13425, 35: 58986, 43: 4594, 44: 6557, 45: 5938, 46: 7392, 47: 4289, 48: 2093, 49: 8069, 50: 10513, 51: 1355, 52: 20675, 53: 2069, 54: 8815, 56: 2252, 57: 2883, 58: 9637, 36: 6949, 59: 3115, 60: 10924, 61: 5631, 62: 11358, 63: 1683, 64: 4102, 55: 4842, 66: 2341, 33: 4029, 67: 3047, 68: 5314, 69: 2729, 70: 3391, 71: 2213, 65: 1397}


[18.267708842208112,
 5.71948608137045,
 11.140771637122004,
 1.0,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 18.43737183868763,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 18.148567103298923,
 20,
 20,
 20,
 20,
 18.123036869068613,
 14.63337063031906,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20,
 20]

In [6]:
# remove troublesome quotes in test data
TEST_DATA = 'test.data'
PROCESSED = 'test.csv'
with open(DATA_FOLDER+TEST_DATA, 'r') as tweets, open(DATA_FOLDER+PROCESSED, 'w') as data:
    buffer = []
    for tweet in tweets:
        buffer.append(tweet)
    for tweet in buffer:
        tweet = tweet.replace("'", ' ')
        tweet = tweet.replace('"', ' ')
        assert('"' not in tweet and "'" not in tweet)
        data.write(tweet)